In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
load_dotenv()

model = ChatOpenAI()


In [36]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str
    feedback: str


In [37]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state


In [38]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state


In [ ]:
def evaluation(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']
    # feedback = state['feedback']

    prompt = f'please exaluate the blog on the title - {title} and the outline {outline} out of 10, not on text only number'

    feedback = model.invoke(prompt).content 

    state['feedback'] = feedback 

    return state
    

In [40]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluation',evaluation)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluation')
graph.add_edge('evaluation', END)

workflow = graph.compile()


In [41]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': "I. Introduction\n    A. Definition of AI\n    B. Introduction to the rise of AI in India\n    C. Importance of AI in the Indian context\n\nII. Historical background of AI in India\n    A. Evolution of AI technology in India\n    B. Major milestones in the development of AI in India\n    C. Key players in the AI industry in India\n\nIII. Current scenario of AI in India\n    A. Growth of AI startups in India\n    B. Adoption of AI technology by Indian businesses\n    C. Government initiatives to promote AI in India\n\nIV. Applications of AI in various industries in India\n    A. Healthcare\n    B. Banking and finance\n    C. Education\n    D. Retail\n    E. Agriculture\n    F. Manufacturing\n\nV. Challenges and opportunities in the AI sector in India\n    A. Talent gap in the AI industry\n    B. Data privacy and security issues\n    C. Ethical considerations in the use of AI\n    D. Opportunities for growth and expansion in the AI sector in In

In [43]:
print(final_state['feedback'])

I would rate this blog outline an 8 out of 10. It covers a comprehensive range of topics related to the rise of AI in India, including historical background, current scenario, applications in various industries, challenges and opportunities, future outlook, and a conclusion. However, it could be improved by including more specific examples of AI technology and startups in India, as well as a deeper analysis of the impact of AI on Indian society and economy. Overall, it provides a good overview of the topic and highlights key areas of interest for readers.
